# TRABALHO 2 - Cenário 3D



 
## Matheus Steigenberg Populim - 1073470
## Vinicius Finke Raiter José - 9791052

Vídeo: https://youtu.be/h1L8l9aoASs<br>

Modelos utilizados: https://drive.google.com/file/d/1DRev6NTqxv_lBBEaODuzpuP9oZeFHzja/view?usp=sharing <br>

### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image

### Inicializando janela

# altura = 900 largura = 1600 <br>
## mude se não for o valor do seu monitor

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.TRUE);
glfw.window_hint(glfw.RESIZABLE, glfw.TRUE);
glfw.window_hint(glfw.CENTER_CURSOR, glfw.TRUE);
#glfw.window_hint(glfw.VISIBLE, glfw.TRUE,glfw.CENTER_CURSOR);
largura = 1600
altura = 900
window = glfw.create_window(largura, altura, "Malhas e Texturas", glfw.get_primary_monitor(), None)
glfw.make_context_current(window)

glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED);


### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec3.
* Definindo matrizes Model, View e Projection que acumulam transformações geométricas 3D e permitem navegação no cenário.
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base nas transformações Model, View e Projection.

In [3]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        varying vec2 out_texture;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [4]:
fragment_code = """
        uniform vec4 color;
        varying vec2 out_texture;
        uniform sampler2D samplerTexture;
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [5]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [6]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [7]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [8]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [9]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [10]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Carregando Modelos (vértices e texturas) a partir de Arquivos

A função abaixo carrega modelos a partir de arquivos no formato WaveFront.


Para saber mais sobre o modelo, acesse: https://en.wikipedia.org/wiki/Wavefront_.obj_file


Nos slides e vídeo-aula da Aula 11 - Parte 1, nós descrevemos o funcionamento desse formato.

In [11]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    texture_coords = []
    faces = []
    material = None
    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])


        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model

In [12]:
glHint(GL_LINE_SMOOTH_HINT, GL_DONT_CARE)
glEnable( GL_BLEND )
glBlendFunc( GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA )
glEnable(GL_LINE_SMOOTH)
glEnable(GL_TEXTURE_2D)
qtd_texturas = 25
textures = glGenTextures(qtd_texturas)
def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    print(img_textura,img.mode)
    img_width = img.size[0]
    img_height = img.size[1]
    #image_data = img.tobytes("raw", "RGB", 0, -1)
    image_data = img.convert("RGBA").tobytes("raw", "RGBA",0,-1)

    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, img_width, img_height, 0, GL_RGBA, GL_UNSIGNED_BYTE, image_data)

### A lista abaixo armazena todos os vertices carregados dos arquivos

In [13]:
vertices_list = []    
textures_coord_list = []

### Vamos carregar cada modelo e definir funções para desenhá-los

## Terreno

In [14]:
modelo = load_model_from_file('modelo/chao_asfalto_grama.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo chao_asfalto_grama.obj. Vertice inicial:',len(vertices_list))
faces_visited = []
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2],' vertice inicial =',len(vertices_list))
        faces_visited.append(face[2])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo chao_asfalto_grama.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(0,'modelo/grass1.jpg')
load_texture_from_file(1,'modelo/road.jpg')

Processando modelo chao_asfalto_grama.obj. Vertice inicial: 0
Material.001  vertice inicial = 0
road.002  vertice inicial = 132
Processando modelo chao_asfalto_grama.obj. Vertice final: 156
modelo/grass1.jpg RGB
modelo/road.jpg RGB


## Céu

In [15]:
modelo = load_model_from_file('modelo/skybox.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo skybox.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo skybox.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(2,'modelo/sky12.jpg')


Processando modelo skybox.obj. Vertice inicial: 156
Processando modelo skybox.obj. Vertice final: 34716
modelo/sky12.jpg RGB


## Casa

In [16]:
modelo = load_model_from_file('modelo/casa.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo casa.obj. Vertice inicial:',len(vertices_list))
faces_visited = []
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2],' vertice inicial =',len(vertices_list))
        faces_visited.append(face[2])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo casa.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(3,'modelo/textura_casa.png')
load_texture_from_file(4,'modelo/chaomadeira.jpg')


Processando modelo casa.obj. Vertice inicial: 34716
Cottage_FREE  vertice inicial = 34716
madeira.001  vertice inicial = 47493
Processando modelo casa.obj. Vertice final: 47511
modelo/textura_casa.png RGB
modelo/chaomadeira.jpg RGB


## Sofá

In [17]:
modelo = load_model_from_file('modelo/sofa2.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo sofa2.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo sofa2.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(5,'modelo/sofacor1.png')

Processando modelo sofa2.obj. Vertice inicial: 47511
Processando modelo sofa2.obj. Vertice final: 97443
modelo/sofacor1.png RGBA


## Escrivaninha

In [18]:
modelo = load_model_from_file('modelo/escrivaninha.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo escrivaninha.obj. Vertice inicial:',len(vertices_list))
faces_visited = []
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2],' vertice inicial =',len(vertices_list))
        faces_visited.append(face[2])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo escrivaninha.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(6,'modelo/madeira1.jpg')
load_texture_from_file(7,'modelo/handle.jpg')


Processando modelo escrivaninha.obj. Vertice inicial: 97443
metalizado  vertice inicial = 97443
macaneta  vertice inicial = 97935
Processando modelo escrivaninha.obj. Vertice final: 98499
modelo/madeira1.jpg RGB
modelo/handle.jpg RGB


## Televisão

In [19]:
modelo = load_model_from_file('modelo/televisao.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo escrivaninha.obj. Vertice inicial:',len(vertices_list))
faces_visited = []
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2],' vertice inicial =',len(vertices_list))
        faces_visited.append(face[2])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo televisao.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas
### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(8,'modelo/black.png')
load_texture_from_file(9,'modelo/william.jpg')

Processando modelo escrivaninha.obj. Vertice inicial: 98499
Material.003  vertice inicial = 98499
Material.001  vertice inicial = 104091
Processando modelo televisao.obj. Vertice final: 104103
modelo/black.png RGB
modelo/william.jpg RGB


## Carro

In [20]:
modelo = load_model_from_file('modelo/carro1.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo carro1.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo carro1.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(10,'modelo/car1.png')


Processando modelo carro1.obj. Vertice inicial: 104103
Processando modelo carro1.obj. Vertice final: 115497
modelo/car1.png RGB


## Carro de Polícia

In [21]:
modelo = load_model_from_file('modelo/carro2.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo carro2.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo carro2.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(11,'modelo/car2.png')


Processando modelo carro2.obj. Vertice inicial: 115497
Processando modelo carro2.obj. Vertice final: 131229
modelo/car2.png RGB


## Árvore

In [22]:
modelo = load_model_from_file('modelo/arvore.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo arvore.obj. Vertice inicial:',len(vertices_list))
faces_visited = []
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2],' vertice inicial =',len(vertices_list))
        faces_visited.append(face[2])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo arvore.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(12,'modelo/bark_0021.jpg')
load_texture_from_file(13,'modelo/DB2X2_L01.png')


Processando modelo arvore.obj. Vertice inicial: 131229
Trank_bark  vertice inicial = 131229
polySurface1SG1  vertice inicial = 151491
Processando modelo arvore.obj. Vertice final: 172401
modelo/bark_0021.jpg RGB
modelo/DB2X2_L01.png RGBA


## Fogueira


In [23]:
modelo = load_model_from_file('modelo/fogueira.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo fogueira.obj. Vertice inicial:',len(vertices_list))
faces_visited = []
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2],' vertice inicial =',len(vertices_list))
        faces_visited.append(face[2])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo fogueira.obj. Vertice final:',len(vertices_list))

load_texture_from_file(14,'modelo/camp_diffuse.jpg')
load_texture_from_file(15,'modelo/woodsground_diffuse.jpg')
load_texture_from_file(16,'modelo/rocks_diffuse.jpg')
load_texture_from_file(17,'modelo/utensilrope_diffuse.jpg')

Processando modelo fogueira.obj. Vertice inicial: 172401
Campfire_Maya_2016:phongE3SG  vertice inicial = 172401
phongE3SG  vertice inicial = 180039
phongE1SG  vertice inicial = 186747
phongE4SG  vertice inicial = 192303
Processando modelo fogueira.obj. Vertice final: 205347
modelo/camp_diffuse.jpg RGB
modelo/woodsground_diffuse.jpg RGB
modelo/rocks_diffuse.jpg RGB
modelo/utensilrope_diffuse.jpg RGB


## Chão "Infinito"

In [24]:
modelo = load_model_from_file('modelo/chao_infinito.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo chao_infinito.obj. Vertice inicial:',len(vertices_list))
faces_visited = []
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2],' vertice inicial =',len(vertices_list))
        faces_visited.append(face[2])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo chao_infinito.obj. Vertice final:',len(vertices_list))

load_texture_from_file(18,'modelo/grass1.jpg')


Processando modelo chao_infinito.obj. Vertice inicial: 205347
Material.002  vertice inicial = 205347
Processando modelo chao_infinito.obj. Vertice final: 205383
modelo/grass1.jpg RGB


## Poço

In [25]:
modelo = load_model_from_file('modelo/well_blender.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo well_blender.obj. Vertice inicial:',len(vertices_list))
faces_visited = []
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2],' vertice inicial =',len(vertices_list))
        faces_visited.append(face[2])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo well_blender.obj. Vertice final:',len(vertices_list))

load_texture_from_file(19,'modelo/well_lower_diffuse.png')
load_texture_from_file(20,'modelo/well_middle_diffuse.png')
load_texture_from_file(21,'modelo/well_roof_diffuse.png')

Processando modelo well_blender.obj. Vertice inicial: 205383
well_lower_texture  vertice inicial = 205383
well_middle_texture  vertice inicial = 208857
well_roof_texture  vertice inicial = 210681
Processando modelo well_blender.obj. Vertice final: 211161
modelo/well_lower_diffuse.png RGB
modelo/well_middle_diffuse.png RGBA
modelo/well_roof_diffuse.png RGB


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar slots.

Nós agora vamos requisitar dois slots.
* Um para enviar coordenadas dos vértices.
* Outros para enviar coordenadas de texturas.

In [26]:
# Request a buffer slot from GPU
buffer = glGenBuffers(2)


###  Enviando coordenadas de vértices para a GPU

In [27]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)


###  Enviando coordenadas de textura para a GPU

In [28]:
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)





### Desenhando nossos modelos
* Cada modelo tem um Model para posicioná-los no mundo.
* É necessário saber qual a posição inicial e total de vértices de cada modelo
* É necessário indicar qual o ID da textura do modelo


## Terreno

In [29]:
def desenha_chao(t_x=0.0,t_y=0.0,t_z=0.0):
    angle = 90.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    t_x=0.0;t_y=0.0;t_z=0.0;
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 0)
    glDrawArrays(GL_TRIANGLES, 0, 132) ## renderizando
    glBindTexture(GL_TEXTURE_2D, 1)
    glDrawArrays(GL_TRIANGLES, 132, 156-132) ## renderizando

## Céu

In [30]:
def desenha_skybox(angle=0.0):
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    t_x = 0.0; t_y = -10.1; t_z = 0.0;
    s_x = 5.0; s_y = 5.0; s_z = 5.0;
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    glBindTexture(GL_TEXTURE_2D, 2)#text. ceu
    glDrawArrays(GL_TRIANGLES, 156, 34716-156) ## renderizando

## Casa

In [31]:
def desenha_casa():
    angle = 270.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    t_x = -18.0; t_y = 0.0; t_z = 0.0;
    s_x = 2.0; s_y = 2.0; s_z = 2.0;
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    glBindTexture(GL_TEXTURE_2D, 3)#textura casa
    glDrawArrays(GL_TRIANGLES, 34716, 47493-34716)
    glBindTexture(GL_TEXTURE_2D, 4)#textura chao casa
    glDrawArrays(GL_TRIANGLES, 47493, 47511-47493)

## Sofá

In [32]:
def desenha_sofa(t_x=0.0,t_y=0.0,t_z=0.0):
    angle = 180.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    #t_x = 5.0; t_y = 1.0; t_z = 3.0;
    s_x = 2.5; s_y = 2.5; s_z = 2.5;
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    glBindTexture(GL_TEXTURE_2D, 5)
    glDrawArrays(GL_TRIANGLES, 47511, 97443-47511) ## renderizando

## Escrivaninha

In [33]:
def desenha_escrivaninha(t_x=-23.0,t_y=2.0,t_z=-9.5):
    angle = 270.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    #t_x = 5.0; t_y = 1.0; t_z = 3.0;
    s_x = 0.5; s_y = 0.5; s_z = 0.5;
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    glBindTexture(GL_TEXTURE_2D, 6)#text. madeira escrivaninha
    glDrawArrays(GL_TRIANGLES, 97443, 97935-97443)
    glBindTexture(GL_TEXTURE_2D, 7)#text. puxador das gavetas
    glDrawArrays(GL_TRIANGLES, 97935, 98499-97935)

## Televisão

In [34]:
def desenha_televisao(t_x=-15.0,t_y=2.0,t_z=-10.5):
    angle = 270.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    #t_x = 5.0; t_y = 1.0; t_z = 3.0;
    s_x = 0.5; s_y = 0.5; s_z = 0.5;
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    glBindTexture(GL_TEXTURE_2D, 8)#text. preta da televisão
    glDrawArrays(GL_TRIANGLES, 98499, 104091-98499)
    glBindTexture(GL_TEXTURE_2D, 9)# william bonner
    glDrawArrays(GL_TRIANGLES, 104091, 104103-104091)

## Carro

In [35]:
def desenha_carro1(t_x=0.0,t_y=0.0,t_z=0.0):
    angle = 90.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    #t_x = 5.0; t_y = 1.0; t_z = 3.0;
    s_x = 1.5; s_y = 1.5; s_z = 1.5;
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    glBindTexture(GL_TEXTURE_2D, 10)
    glDrawArrays(GL_TRIANGLES, 104103, 115497-104103) ## renderizando

## Carro de Polícia

In [36]:
def desenha_carro2(t_x=0.0,t_y=0.0,t_z=0.0):
    angle = 90.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    #t_x = 5.0; t_y = 1.0; t_z = 3.0;
    s_x = 1.5; s_y = 1.5; s_z = 1.5;
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    glBindTexture(GL_TEXTURE_2D, 11)
    glDrawArrays(GL_TRIANGLES, 115497, 131229-115497) ## renderizando

## Árvore

In [37]:
def desenha_arvore():
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    t_x = -20.0; t_y = -1.0; t_z = -40.0;
    s_x = 7.0; s_y = 7.0; s_z = 7.0;
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    glBindTexture(GL_TEXTURE_2D, 12)#text. tronco
    glDrawArrays(GL_TRIANGLES, 131229, 151491-131229) ## renderizando
    glBindTexture(GL_TEXTURE_2D, 13)#text. folhas
    glDrawArrays(GL_TRIANGLES, 151491, 172401-151491) ## renderizando

## Fogueira e Barraca

In [38]:
def desenha_fogueira():
    angle = 45.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    t_x = -20.0; t_y = 0.2; t_z = 50.0;
    s_x = 0.2; s_y = 0.2; s_z = 0.2;
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    glBindTexture(GL_TEXTURE_2D, 14)#text. tronco
    glDrawArrays(GL_TRIANGLES, 172401, 180039-172401) ## renderizando
    glBindTexture(GL_TEXTURE_2D, 15)#text. folhas
    glDrawArrays(GL_TRIANGLES, 180039, 186747-180039) ## renderizando
    glBindTexture(GL_TEXTURE_2D, 16)#text. tronco
    glDrawArrays(GL_TRIANGLES, 186747, 192303-186747) ## renderizando
    glBindTexture(GL_TEXTURE_2D, 17)#text. folhas
    glDrawArrays(GL_TRIANGLES, 192303, 205347-192303) ## renderizando

## Chão "Infinito"

In [39]:
def desenha_chao_infinito():
    angle = 0.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    t_x = 0.0; t_y = -10.0; t_z = 0.0;
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    glBindTexture(GL_TEXTURE_2D, 18)#text. grama
    glDrawArrays(GL_TRIANGLES, 205347, 205383-205347) ## renderizando

## Poço

In [40]:
def desenha_poco():
    angle = 0.0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    t_x = 30.0; t_y = -0.3; t_z = 20.0;
    s_x = 0.3; s_y = 0.3; s_z = 0.3;
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    glBindTexture(GL_TEXTURE_2D, 19)#text. grama
    glDrawArrays(GL_TRIANGLES, 205383, 208857-205383) ## renderizando
    glBindTexture(GL_TEXTURE_2D, 20)#text. grama
    glDrawArrays(GL_TRIANGLES, 208857, 210681-208857) ## renderizando
    glBindTexture(GL_TEXTURE_2D, 21)#text. grama
    glDrawArrays(GL_TRIANGLES, 210681, 211161-210681) ## renderizando

### Eventos para modificar a posição da câmera.

* Usei as teclas A, S, D e W para movimentação no espaço tridimensional
* Usei a posição do mouse para "direcionar" a câmera

In [41]:
cameraPos   = glm.vec3(0.0,  4.0,  1.0);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);


polygonal_mode = False

def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode
    cameraSpeed = 0.8
    if key == 87 and (action==1 or action==2): # tecla W
        cameraPos += cameraSpeed * cameraFront
    if key == 83 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraFront
    if key == 65 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
    if key == 68 and (action==1 or action==2): # tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
    if key == 32 and (action==1 or action==2): # tecla espaço
        cameraPos += cameraSpeed * cameraUp
    if key == 90 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraUp
    if key == 80 and action==1 and polygonal_mode==True:
        polygonal_mode=False
    else:
        if key == 80 and action==1 and polygonal_mode==False:
            polygonal_mode=True
        
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos
    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0
    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)
    
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)

### Matrizes Model, View e Projection

Teremos uma aula específica para entender o seu funcionamento.

In [42]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    angle = math.radians(angle)
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade
    # aplicando translacao
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z)) 
    # aplicando rotacao
    matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))
    # aplicando escala
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z)) 
    matrix_transform = np.array(matrix_transform).T # pegando a transposta da matriz (glm trabalha com ela invertida)
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(90.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Nesse momento, nós exibimos a janela!


In [43]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

In [44]:
glEnable(GL_DEPTH_TEST) ### importante para 3D

z_carro1 = 30
z_carro2 = 0

while not glfw.window_should_close(window):
    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    #controla e aplica o estado do modo poligonal
    if polygonal_mode==True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode==False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
    
    #verificação pra câmera não sair do cenário
    if(abs(cameraPos[0])>40):
        cameraPos[0] = 40*cameraPos[0]/abs(cameraPos[0])
    if(abs(cameraPos[2])>80):
        cameraPos[2] = 80*cameraPos[2]/abs(cameraPos[2])
    
    #verificação pra câmera não passar do chão
    if(cameraPos[1] < 4.0):
        cameraPos[1] = 4.0
    
    #verificação pra câmera não sair da esfera que forma o céu
    x = cameraPos[0]
    y = cameraPos[1]
    z = cameraPos[2]
    if((x/500)**2+((y+10)/500)**2+(z/500)**2 > 1):
        cameraPos = glm.vec3(0.0,  4.0,  1.0);

    #movimentação dos carros
    z_carro1 += 40/60.0
    if(z_carro1 > 80):
        z_carro1 = -80
    z_carro2 += 40/60.0
    if(z_carro2 > 80):
        z_carro2 = -80

    #desenhando todos os objetos do cenário:
    #algumas funções de desenho estão com argumentos para facilitar ajustes
    desenha_chao_infinito()
    desenha_chao()
    desenha_casa()
    desenha_arvore()
    desenha_skybox()
    desenha_escrivaninha(-23.0,2.0,-8.8)
    desenha_sofa(-16,2,10)
    desenha_televisao()
    desenha_carro1(3,0.1,z_carro1)
    desenha_carro2(3,0.1,z_carro2)
    desenha_fogueira()
    desenha_poco()

    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_FALSE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_FALSE, mat_projection)    
    
    glfw.swap_buffers(window)

glfw.terminate()

-25.95245361328125 488.8380126953125 28.775617599487305
